In [ ]:

import sys
sys.path.append("././quaos")
from math import gcd
from collections import defaultdict
from itertools import product

import numpy as np
from hamiltonian import random_pauli_hamiltonian, cancel_pauli, symplectic_pauli_reduction, pauli_reduce
from paulis import (
    PauliSum, PauliString, Pauli,
    #  Xnd, Ynd, Znd, Id, symplectic_to_string,
    string_to_symplectic,
)
from circuits.utils import solve_modular_linear

In [ ]:

class Gate:
    """
    Mapping can be written as set of rules,
    
    e.g. for CNOT
                x1z0*x0z0 -> x1z0*x1z0
                x0z0*x1z0 -> x0z0*x1z0  # doesn't need specifying
                x0z1*x0z0 -> x0z1*x0z0  # doesn't need specifying
                x0z0*x0z1 -> x0z-1*x0z1 # 

    inputs are:

    mapping = ['x1z0*x0z0 -> x1z0*x1z0', 'x0z0*x0z1 -> -x0z1*x0z1']  # (control*target -> control*target)

    """
    def __init__(self, name: str, qudit_indices: list[int], mapping: list[str], dimension: list[int]):
        self.dimension = dimension
        self.name = name
        self.qudit_indices = qudit_indices
        self.mapping = mapping
        self.n_qudits = len(qudit_indices)
        self.symplectic = self._interpret_mapping(mapping)
        
    
    def _interpret_mapping(self, map_string: list[str]) -> np.ndarray:
        map_from, map_to = zip(*[map_string[i].split('->') for i in range(len(map_string))])

        n_maps = len(map_from)

        symplectic_looked_for = []
        for i in range(n_maps):
            s, _ = string_to_symplectic(map_from[i])
            symplectic_looked_for.append(s)

        symplectic_mapped_to = []
        acquired_phase = []
        for i in range(len(map_from)):
            s, p = string_to_symplectic(map_to[i])
            symplectic_mapped_to.append(s)
            acquired_phase.append(p)
        
        # For an n qubit operation there should be a set of 2n mappings.
        # If the rest are unspecified, they should be identity mappings (those with equal looked_for and mapped_to)
        # If the user specifies less than 2n mappings, then the remaining ones will be identity mappings
        # These remaining identity mappings must all be linearly independent of the specified ones
        # note that this will fail if the input mappings are not of the form X*I*...*I, Z*I*...*I, (having only a single
        # X or Z in the string - checked for below) - generalisable if needed
        # This will only be needed to obtain the symplectic matrix for the gate operation, not for current method...
        if len(symplectic_looked_for) < 2 * self.n_qudits:
            for s in symplectic_looked_for:
                if np.sum(s) != 1:
                    raise Exception('Unable to automate completion of mappings. Specify 2*n_qudits linearly '
                                    'independent mappings to fully define the gate operation.')
            for i in range(2 * self.n_qudits):
                mapping = np.zeros(2 * self.n_qudits, dtype=int)
                mapping[i] = 1
                if not any(np.array_equal(mapping, arr) for arr in symplectic_looked_for):
                    print(mapping)
                    symplectic_looked_for.append(mapping)
                    symplectic_mapped_to.append(mapping)

        ## remove once debugged
        # assert len(symplectic_looked_for) == self.dimension * (self.n_qudits), (len(symplectic_looked_for),
        #                                                                              self.dimension * (self.n_qudits))
        # assert len(symplectic_mapped_to) == self.dimension * (self.n_qudits)
        ##

        symplectic_looked_for = np.array(symplectic_looked_for)
        symplectic_mapped_to = np.array(symplectic_mapped_to)

        # reorder such that the symplectic looked for is always the identity
        # (this would need to be altered to an arbitrary complete set generalise)
        perm = np.argmax(symplectic_looked_for, axis=1)
        inverse_perm = np.argsort(perm)
        symplectic_looked_for = symplectic_looked_for[inverse_perm]
        symplectic_mapped_to = symplectic_mapped_to[inverse_perm]

        symplectic = self.build_symplectic_from_mappings(symplectic_looked_for, symplectic_mapped_to)

        return symplectic
    
    def mod_inv(self, a):
        """Modular inverse of a mod d, where d = self.dimension."""
        a = a % self.dimension
        if a == 0:
            raise ValueError("0 has no inverse modulo d")
        return pow(int(a), -1, self.dimension)

    def mod_mat_inv(self, A):
        """Modular inverse of matrix A over Z_d using Gauss-Jordan elimination."""
        A = np.array(A, dtype=int) % self.dimension
        n = A.shape[0]
        I = np.eye(n, dtype=int)
        AI = np.hstack([A, I])  # Augmented matrix [A | I]

        for i in range(n):
            # Find pivot
            pivot = AI[i, i]
            if pivot == 0:
                # Try to swap with a lower row
                for j in range(i + 1, n):
                    if AI[j, i] != 0:
                        AI[[i, j]] = AI[[j, i]]
                        pivot = AI[i, i]
                        break
                else:
                    raise ValueError(f"Matrix {A} not invertible")

            # Normalize pivot row
            inv_pivot = self.mod_inv(pivot)
            AI[i] = (AI[i] * inv_pivot) % self.dimension

            # Eliminate other rows
            for j in range(n):
                if j != i:
                    factor = AI[j, i]
                    AI[j] = (AI[j] - factor * AI[i]) % self.dimension

        A_inv = AI[:, n:]  # Extract right half
        return A_inv

    def build_symplectic_from_mappings(self, looked_for, mapped_to):
        """Build symplectic matrix from Pauli generator mappings."""
        looked_for = np.array(looked_for, dtype=int) % self.dimension
        mapped_to = np.array(mapped_to, dtype=int) % self.dimension

        if looked_for.shape != mapped_to.shape:
            raise ValueError("Shape mismatch between looked_for and mapped_to.")
        # if looked_for.shape[0] != looked_for.shape[1]:
        #     raise ValueError("Expect square matrix of 2n symplectic vectors.", looked_for)

        inv_looked_for = self.mod_mat_inv(looked_for)
        S = (mapped_to @ inv_looked_for) % self.dimension
        return S


class SUM(Gate):
    def __init__(self, control, target, dimension):
        SGate_operations = self.sum_gate_operations(dimension)
        super().__init__("SUM", [control, target], SGate_operations, dimension=dimension)
   
    @staticmethod
    def sum_gate_operations(dimension: int) -> list[str]:
        operations = []
        for r1 in range(dimension):
            for s1 in range(dimension):
                for r2 in range(dimension):
                    for s2 in range(dimension):
                        new_r1 = r1
                        new_s1 = (s1 - s2) % dimension
                        new_r2 = (r2 + r1) % dimension
                        new_s2 = s2
                        phase = (r1 * s2) % dimension
                        if r1 != new_r1 or s1 != new_s1 or r2 != new_r2 or s2 != new_s2 or phase != 0:
                            operations.append(f"x{r1}z{s1} x{r2}z{s2} -> x{new_r1}z{new_s1} x{new_r2}z{new_s2}p{phase}")
        print( np.array(operations).T)
        print(len(operations))
        return operations

class SUMcs(Gate):
    def __init__(self, control, target, dimension):
        SGate_operations = self.sum_gate_operations(dimension)
        super().__init__("SUM", [control, target], SGate_operations, dimension=dimension)
   
    @staticmethod
    def sum_gate_operations(dimension: int) -> list[str]:
        operations = []
        for r1 in range(dimension):
            for s1 in range(dimension):
                for r2 in range(dimension):
                    for s2 in range(dimension):
                        new_r1 = r1
                        new_s1 = (s1 - s2) % dimension
                        new_r2 = (r2 + r1) % dimension
                        new_s2 = s2
                        phase = (r1 * s2) % dimension
                        if r1 + r2 + s1 + s2 == 1:
                            operations.append(f"x{r1}z{s1} x{r2}z{s2} -> x{new_r1}z{new_s1} x{new_r2}z{new_s2}p{phase}")
        print( np.array(operations).T)
        print(len(operations))
        return operations
    
    @staticmethod
    def sum_gate_images() -> list[np.ndarray]:
        images = [np.array([1, 1, 0, 0]),  # image of X0:  X0 -> X0 X1
                  np.array([0, 1, 0, 0]),  # image of X1:  X1 -> X1
                  np.array([0, 0, 1, 0]),  # image of Z0:  Z0 -> Z0
                  np.array([0, 0, -1, 1])  # image of Z1:  Z1 -> Z0^-1 Z1
                  ]
        return images

In [ ]:
sum_cs = SUMcs(1, 0, 2)
# sum_gate = SUM(0, 1, 2)
sum_cs.symplectic 
xi_symplectic = np.array([1, 0, 0, 0])
ix_symplectic = np.array([0, 1, 0, 0])
zi_symplectic = np.array([0, 0, 1, 0])
iz_symplectic = np.array([0, 0, 0, 1])
# test_symplectic = np.array([1, 2, 2, 1])

print(sum_cs.symplectic)
sum_cs.symplectic[3, 2] = -1
print(sum_cs.symplectic)

print((sum_cs.symplectic @ xi_symplectic )% 2) 
print((sum_cs.symplectic @ ix_symplectic)% 2) 
print((sum_cs.symplectic @ zi_symplectic)% 2) 
print((sum_cs.symplectic @ iz_symplectic )% 2) 
# print((sum_cs.symplectic.T @ test_symplectic @ sum_cs.symplectic)% 3) 



In [ ]:
def str_to_int(string):
    output = []

    for s in string:
        if s == 'I':
            output.append(0)
        elif s == 'X':
            output.append(1)
        elif s == 'Y':
            output.append(2)
        elif s == 'Z':
            output.append(3)
    return output

def int_to_pauli(integer):
    if integer == 0:
        return [0, 0]
    elif integer == 1:
        return [1, 0]
    elif integer == 2:
        return [1, 1]
    elif integer == 3:
        return [0, 1]

def matrix_to_pauli_sum(matrix, weights, phases, dimensions):

    n_paulis = matrix.shape[0]
    n_qudits = matrix.shape[1]

    pauli_strings = []
    for i in range(n_paulis):
        x_exp = np.zeros(n_qudits, dtype=int)
        z_exp = np.zeros(n_qudits, dtype=int)
        for j in range(n_qudits):
            x_exp[j] = int_to_pauli(matrix[i, j])[0]
            z_exp[j] = int_to_pauli(matrix[i, j])[1]
        ps = PauliString(x_exp, z_exp, dimensions)
        pauli_strings.append(ps)

    return PauliSum(pauli_strings, weights, phases, dimensions, standardise=False)
    


def find_allowed_target(pauli_sum, target_pauli_list, ):
    pauli_list = [target_pauli_list[i][0] for i in range(len(target_pauli_list))]
    pauli_list = str_to_int(pauli_list)
    string_indices = [target_pauli_list[i][1] for i in range(len(target_pauli_list))]
    qudit_indices = [target_pauli_list[i][2] for i in range(len(target_pauli_list))]

    dims = pauli_sum.dimensions 
    combined_indices = list(zip(string_indices, qudit_indices))
    index_dict = defaultdict(list)

    for idx, tup in enumerate(combined_indices):
        index_dict[tup].append(idx)

    underdetermined_pauli_indices = [combined_indices[idxs[0]] for idxs in index_dict.values() if len(idxs) > 1]
    underdetermined_pauli_options = []
    for indices in underdetermined_pauli_indices:
        options = []
        for i in range(len(target_pauli_list)):
            if target_pauli_list[i][1] == indices[0] and target_pauli_list[i][2] == indices[1]:
                if target_pauli_list[i][0] not in options:
                    options.append(target_pauli_list[i][0])
        underdetermined_pauli_options.append(options)
    underdetermined_pauli_options = [str_to_int(l) for l in underdetermined_pauli_options]

    determined_pauli_indices = [combined_indices[idxs[0]] for idxs in index_dict.values() if len(idxs) == 1]

    options_matrix = np.empty([pauli_sum.n_paulis(), pauli_sum.n_qudits()], dtype=object)
    for i in range(pauli_sum.n_paulis()):
        for j in range(pauli_sum.n_qudits()):
            if (i, j) in underdetermined_pauli_indices:
                options_matrix[i, j] = underdetermined_pauli_options[underdetermined_pauli_indices.index((i, j))]
            elif (i, j) in determined_pauli_indices:
                options_matrix[i, j] = [pauli_list[determined_pauli_indices.index((i, j))]]
            else:
                options_matrix[i, j] = [0, 1, 2, 3]
    
    flag_matrix = np.zeros([pauli_sum.n_paulis(), pauli_sum.n_qudits()], dtype=int)  # flag matrix to track which indices are determined 
    # 0 for not determined, 1 for underdetermined, -1 for determined
    for idx, tup in enumerate(combined_indices):
        if tup in underdetermined_pauli_indices:
            flag_matrix[tup[0], tup[1]] = -1  # -1 for underdetermined
        else:
            flag_matrix[tup[0], tup[1]] = 1  # 1 for determined

    spm = pauli_sum.symplectic_product_matrix()
    
    possible_targets = []
    for combo in product(*options_matrix.flatten()):
        combo = np.reshape(combo, (pauli_sum.n_paulis(), pauli_sum.n_qudits()))
        pauli_sum_candidate = matrix_to_pauli_sum(combo, pauli_sum.weights, pauli_sum.phases, dims)
        candidate_spm = pauli_sum_candidate.symplectic_product_matrix()

        if np.all(spm == candidate_spm):
            possible_targets.append(pauli_sum_candidate)
    return possible_targets



def is_ix(pauli_string: PauliString, qudit: int | None = None) -> bool:
    if qudit is None:
        if np.any(pauli_string.z_exp != 0):
            return False
        if np.count_nonzero(pauli_string.x_exp) == 1:
            return True
        else:
            return False
    else:
        if pauli_string.z_exp[qudit] != 0:
            return False
        if pauli_string.x_exp[qudit] >= 1 and np.count_nonzero(pauli_string.x_exp) == 1:
            return True
        else:
            return False


def is_iz(pauli_string: PauliString, qudit: int | None = None) -> bool:
    if qudit is None:
        if np.any(pauli_string.x_exp != 0):
            return False
        if np.count_nonzero(pauli_string.z_exp) == 1:
            return True
        else:
            return False
    else:
        if pauli_string.x_exp[qudit] != 0:
            return False
        if pauli_string.z_exp[qudit] >= 1 and np.count_nonzero(pauli_string.z_exp) == 1:
            return True
        else:
            return False


def find_ix_iz(pauli_sum: PauliSum, qudit: int | None = None) -> tuple[list[int], list[int]]:
    ixs = []
    izs = []
    for i in range(pauli_sum.n_paulis()):
        if is_ix(pauli_sum.pauli_strings[i], qudit):
            ixs.append(i)
        elif is_iz(pauli_sum.pauli_strings[i], qudit):
            izs.append(i)
    return ixs, izs


def use_ix_remove_x(pauli_sum: PauliSum, ixs: list[int]):
    new_ps = pauli_sum.copy()
    multiplied_paulis = []
    for ix in ixs:  # the ixth string is the ix - multiply others by this to remove their x terms on x_qudit
        x_qudit = np.where(new_ps[ix].x_exp != 0)[0][0]
        x_exp = new_ps[ix].x_exp[x_qudit]
        for i in range(pauli_sum.n_paulis()):
            if i != ix:
                if pauli_sum[i].x_exp[x_qudit] != 0:
                    n = solve_modular_linear(pauli_sum[i].x_exp[x_qudit], x_exp, pauli_sum.dimensions[x_qudit])
                    new_ps[i] = pauli_sum[i] * pauli_sum[ix]**n
                multiplied_paulis.append(())


def standard_form_to_basis(pauli_sum: PauliSum) -> tuple[PauliSum, list[tuple[int, int, int]]]:
    new_ps = pauli_sum.copy()
    multiplied_paulis = []

    for q in range(pauli_sum.n_qudits()):
    
        ixs, izs = find_ix_iz(new_ps, q)
        if len(ixs) == 0 and len(izs) == 0:
            print('No ix or iz found for qudit ', q)
            continue
        ixs = ixs[0] if len(ixs) > 0 else None
        izs = izs[0] if len(izs) > 0 else None
        if ixs is None:
            print('No ix found for qudit ', q)
        if izs is None:
            print('No iz found for qudit ', q)
        for p in range(new_ps.n_paulis()):  # min( , 2 * new_ps.n_qudits())
            if new_ps[p].x_exp[q] != 0 and ixs is not None and ixs != p:
                n = solve_modular_linear(new_ps[p].x_exp[q], new_ps[ixs].x_exp[q], new_ps.dimensions[q])
                new_ps[p] = new_ps[p] * new_ps[ixs]**n
                multiplied_paulis.append((p, ixs, n))
            if new_ps[p].z_exp[q] != 0 and izs is not None and izs != p:
                n = solve_modular_linear(new_ps[p].z_exp[q], new_ps[izs].z_exp[q], new_ps.dimensions[q])
                new_ps[p] = new_ps[p] * new_ps[izs]**n
                multiplied_paulis.append((p, izs, n))
    return new_ps, multiplied_paulis


def multiply_paulis(pauli_sum: PauliSum, multiplier_list: list[tuple[int, int, int]]) -> PauliSum:
    """
    Multiply the pauli strings in the pauli sum by the given multipliers.

    :param pauli_sum: The PauliSum to multiply.
    :param multiplier_list: A list of tuples (pauli_index, multiplier_index, multiplier_value) where
    the pauli at pauli_index is multiplied by the pauli at multiplier_index raised to the power of multiplier_value.

    :return: The PauliSum after multiplication.
    """
    new_pauli_sum = pauli_sum.copy()
    for p, m, n in multiplier_list:
        new_pauli_sum[p] = new_pauli_sum[p] * new_pauli_sum[m]**n
    return new_pauli_sum


def is_basis(pauli_sum: PauliSum) -> tuple[bool, list[int]]:
    ixs, izs = find_ix_iz(pauli_sum)
    if len(ixs) + len(izs) == 2 * pauli_sum.n_qudits():
        return True, ixs + izs
    elif len(ixs) + len(izs) > 2 * pauli_sum.n_qudits():
        # over complete - pick only the first 2 * n_qudits independent pauli strings
        ix_qudits = []
        ixs_ = []
        iz_qudits = []
        izs_ = []
        for ix in ixs:
            ixq = np.where(pauli_sum[ix].x_exp != 0)[0][0]
            if ixq not in ix_qudits:
                ix_qudits.append(ixq)
                ixs_.append(ix)
        for iz in izs:
            izq = np.where(pauli_sum[iz].z_exp != 0)[0][0]
            if izq not in iz_qudits:
                iz_qudits.append(izq)
                izs_.append(iz)
        return True, ixs_ + izs_
    else:
        # Check if the remaining pauli strings can be made up of the ixs and izs - if so it is an incomplete basis
        remaining = [i for i in range(pauli_sum.n_paulis()) if i not in ixs and i not in izs]
        missing_ix = []
        missing_iz = []
        for q in range(pauli_sum.n_qudits()):
            ixs_q, izs_q = find_ix_iz(pauli_sum, q)
            if len(ixs_q) == 0:
                missing_ix.append(q)
            if len(izs_q) == 0:
                missing_iz.append(q)
        for r in remaining:
            r_x_exp = pauli_sum[r].x_exp
            r_z_exp = pauli_sum[r].z_exp
            for q in missing_ix:
                if r_x_exp[q] != 0:
                    # If the pauli string has an x term on a qudit where we are missing an ix, it cannot be a basis
                    return False, []
            for q in missing_iz:
                if r_z_exp[q] != 0:
                    # If the pauli string has a z term on a qudit where we are missing an iz, it cannot be a basis
                    return False, []
        return True, ixs + izs


In [ ]:
init_sum = ['x1z0 x0z0', 'x1z0 x0z1', 'x0z1 x1z1']
init_sum = PauliSum(init_sum, dimensions=[2, 2])
print(init_sum)
target_paulis = [('X', 0, 0), ('I', 0, 1), ('Z', 1, 0), ('I', 1, 0), ('Z', 2, 0), ('I', 2, 0)]  #

targets = find_allowed_target(init_sum, target_paulis)

print(len(targets))
for t in targets:
    print(t)

In [ ]:

def get_gate_from_target(input_pauli_sum: PauliSum, target_pauli_sum: PauliSum) -> Gate:
    """
    Given an input PauliSum and a target PauliSum, find the gate that maps the input to the target.

    We assumer that the input_pauli_sum is in standard form, that is, it is one of the conditioned PauliSums output 
    by the symplectic reduction method.
    """

    if np.any(input_pauli_sum.symplectic_product_matrix() != target_pauli_sum.symplectic_product_matrix()):
        raise ValueError("Input and target PauliSums must have the same symplectic product matrix.")
    
    # Find linear operations that map the input PauliSum to a basis representation
    
    input_pauli_basis, multipliers = standard_form_to_basis(input_pauli_sum)
    target_pauli_basis = multiply_paulis(target_pauli_sum, multipliers)

    # We then use this basis representation to construct the gate mappings to C^dag target C plus same linear operations 
    basis_check, basis_indices = is_basis(input_pauli_basis)
    if not basis_check:
        raise ValueError("Output PauliSum is not a basis representation.")
    gate_mappings = []
    for i in range(len(basis_indices)):
        gate_mappings.append((basis_indices[i], target_pauli_basis[i]))

    return Gate("CustomGate", list(range(input_pauli_sum.n_qudits())), gate_mappings, input_pauli_sum.dimensions)

In [ ]:
CGate = get_gate_from_target(init_sum, targets[0])
print(CGate)